In [28]:
import random
import math
import numpy as np
from math import radians, cos, sin, asin, sqrt

In [27]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [26]:
def customPow(x):
    return pow(x,-0.86)

def genRankSelectionCDF(n):
    rank_select_pdf=np.array(range(1,n+1))
    rank_select_pdf=map(customPow,rank_select_pdf)
    pdf_sum=sum(rank_select_pdf)
    rank_select_pdf/=pdf_sum
    rank_select_cdf=np.cumsum(rank_select_pdf)
    return rank_select_cdf

In [76]:
rank_select_cdf=genRankSelectionCDF(100000)

In [109]:
#loc_pool contains the lon,lat of all the potential destinations
#locs: locid, visit time, lon, lat, locid start with 
def findNextLoc(locs,curr_loc,loc_pool,rho,gamma,olon,olat,rank_select_cdf,min_dist):
    visited_location=len(locs)
    r_return_unconditioned = 0.6*(1-rho*pow(visited_location,gamma))
    p_return=r_return_unconditioned/(r_return_unconditioned+rho*pow(visited_location,gamma))
    if random.random()<p_return:
        #return
        visit_count_sum=0.0
        for j in range(1,visited_location):
            if curr_loc!=j:
                visit_count_sum+=locs[j][1]
        visit_cdf=[0]*visited_location
        #if curr_loc!=0:
            #visit_cdf[0]=0
        for j in range(1,visited_location):
            if curr_loc!=j:
                visit_cdf[j]=visit_cdf[j-1]+locs[j][1]/visit_count_sum
            else:
                visit_cdf[j]=visit_cdf[j-1]
        found_sign=0
        while found_sign==0:
            rand_num=random.random()
            for j in range(visited_location):
                if rand_num<visit_cdf[j] and j!=curr_loc:
                    locs[j][1]+=1
                    return j
    #new location, choose from the loc pool using rank
    n=len(loc_pool)
    # a list of [dist,loc id]
    dists= [[] for x in xrange(n)]
    for i in range(n):
        dists[i]=[haversine(loc_pool[i][0], loc_pool[i][1], olon, olat), i]
    dists.sort()
    
    for i in range(n):
        if dists[i][0]>min_dist:
            begin_index=i
            break
    
    find_sign=0
    while find_sign==0:
        rand_num=random.random()
        for i in range(begin_index,n):
            if rand_num<rank_select_cdf[i-begin_index]:
                find_sign=1
                locs.append([locs[-1][0]+1,1,loc_pool[dists[i][1]][0],loc_pool[dists[i][1]][1]])
                if abs(loc_pool[dists[i][1]][0]+71.107221)<0.0001:
                    print i, dists[i], locs[-1][0]
                
                return locs[-1][0]

def loadLocPool():
    f = open('OtherLocation.txt','r')
    loc_pool = [[float(x.split(" ")[1]), float(x.split(" ")[0])] for x in f.readlines()]
    return loc_pool

#the input information locs is: locid, visit time, lon, lat, 
#parameters needed: b1, b2, start slot, end slot
#home is locid 0, other location id starts at 1
def interRec(locs,b1,b2,start_slot,end_slot,nw,rho,gamma,min_dist,loc_pool):
    #load daily_activeness
    f = open('DailyTrend.txt', 'r')
    daily_activeness = map(float,f.readlines()[0].split(' '))
    #load weekly_activeness
    f = open('WeeklyTrend.txt', 'r')
    weekly_activeness = map(float,f.readlines()[0].split(' '))
    
    slot_num=end_slot-start_slot+1
    person_loc=[0]*slot_num
    at_home=1
    curr_loc=0
    for i in range(slot_num):
        curr_slot=start_slot+i
        daily_slot=curr_slot%144
        weekly_slot=(curr_slot/144)%7
        if at_home:
            pt=nw*daily_activeness[daily_slot]*weekly_activeness[weekly_slot]
            #from home to other
            if random.random()<pt:
                at_home=0
                #decide where to go
                person_loc[i]=findNextLoc(locs,curr_loc,loc_pool,rho,gamma,locs[curr_loc][2],locs[curr_loc][3],rank_select_cdf,min_dist)
                curr_loc=person_loc[i]
            else:
                #keep at home, record it
                person_loc[i]=0
                curr_loc=0
        else:
            #p_other_move=getOtherMovePr(daily_slot,weekly_slot,b1,nw)
            p_other_move = b1*nw*daily_activeness[daily_slot]*weekly_activeness[weekly_slot];
            if random.random()<p_other_move:
                #move to home or another other
                #p_other_home=getOtherMoveToHomePr(daily_slot,weekly_slot,b2,nw);
                p_other_home=1-b2*nw*daily_activeness[daily_slot]*weekly_activeness[weekly_slot];
                if random.random()<p_other_home:
                    at_home=1
                    person_loc[i]=0
                    curr_loc=0
                else:
                    #other to other
                    #decide where to go
                    print len(locs),curr_loc
                    person_loc[i]=findNextLoc(locs,curr_loc,loc_pool,rho,gamma,locs[curr_loc][2],locs[curr_loc][3],rank_select_cdf,min_dist)
                    curr_loc=person_loc[i]
            else:
                #else keep at the current other place
                person_loc[i]=curr_loc
    return locs, person_loc
        
        

In [114]:
f = open('SparseInput.txt', 'r')
locs = [map(float,x.split(' ')) for x in f.readlines()]
loc_pool = loadLocPool()
for i in range(len(locs)):
    locs[i][0] = int(locs[i][0])
locs, person_loc = interRec(locs,4,23,200,6000,6,0.6,-0.21,0.6,loc_pool)

8 7
12 11
15 13
16 15
18 8


In [115]:
#process person_loc to be the same format as the matlab code
loc_freq=[0]*100
stays=[]
for i in range(len(person_loc)):
    if i==0 or person_loc[i]!=person_loc[i-1]:
        #a new stay
        loc_freq[person_loc[i]]+=1
        day=int(i/144)
        lat=locs[person_loc[i]][3]
        lon=locs[person_loc[i]][2]
        hour=(i%144)/6.0
        visits=loc_freq[person_loc[i]]
        stays.append([0,0,0,day,person_loc[i]+1,lat,lon,hour,visits])

In [116]:
stays

[[0, 0, 0, 0, 1, 42.394486, -71.121227, 0.0, 1],
 [0, 0, 0, 0, 2, 42.357804, -71.099643, 7.333333333333333, 1],
 [0, 0, 0, 0, 1, 42.394486, -71.121227, 8.833333333333334, 2],
 [0, 0, 0, 2, 5, 42.390594, -71.126292, 13.833333333333334, 1],
 [0, 0, 0, 3, 1, 42.394486, -71.121227, 2.5, 3],
 [0, 0, 0, 3, 6, 42.400874, -71.12296, 20.666666666666668, 1],
 [0, 0, 0, 4, 1, 42.394486, -71.121227, 4.0, 4],
 [0, 0, 0, 4, 5, 42.390594, -71.126292, 10.166666666666666, 2],
 [0, 0, 0, 5, 1, 42.394486, -71.121227, 9.5, 5],
 [0, 0, 0, 6, 7, 42.399899, -71.121728, 6.833333333333333, 1],
 [0, 0, 0, 7, 1, 42.394486, -71.121227, 1.0, 6],
 [0, 0, 0, 10, 8, 42.399627, -71.123717, 7.333333333333333, 1],
 [0, 0, 0, 10, 9, 42.398535, -71.140045, 23.0, 1],
 [0, 0, 0, 11, 1, 42.394486, -71.121227, 8.666666666666666, 7],
 [0, 0, 0, 11, 10, 42.399958, -71.121455, 21.5, 1],
 [0, 0, 0, 12, 1, 42.394486, -71.121227, 1.6666666666666667, 8],
 [0, 0, 0, 12, 11, 42.396284, -71.14143, 6.0, 1],
 [0, 0, 0, 12, 1, 42.394486, 

In [117]:
with open("SparseUser.txt", "w") as text_file:
    for stay in stays:
        text_file.write("%d %d %d %d %d %f %f %f %d\n" % tuple(stay))